In [15]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd

In [16]:
app = Flask(__name__)

In [17]:
# Load the trained model
with open('price_prediction_model.pkl', 'rb') as model_file:
    price_prediction_model = pickle.load(model_file)

In [18]:
with open('motor_scaler.pkl', 'rb') as scaler_file:
    motor_scaler = pickle.load(scaler_file)
   
with open('price_scaler.pkl', 'rb') as scaler_file:
    price_scaler = pickle.load(scaler_file)

In [19]:
training_data = pd.read_csv('resultDF.csv')

In [20]:
excluded_columns = ['Price', 'normalized_price'] 
feature_names = [col for col in training_data.columns if col not in excluded_columns]

In [21]:
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Initialize DataFrame with default values (zeros)
        default_data = pd.DataFrame([[0] * len(feature_names)], columns=feature_names)

        # Get data from the request
        data = request.get_json(force=True)
        app.logger.info(f"Received data: {data}")

        # Convert received data to DataFrame
        input_data = pd.DataFrame([data])

        # Extract features used during training
        features_used_during_training = input_data.columns

        # Select only the relevant features from the received data
        input_data_selected = input_data[features_used_during_training]

        # Update the default_data with the user input
        default_data.update(input_data_selected)

        # Scale the 'Motor power' feature using the motor scaler
        default_data['Motor power'] = motor_scaler.transform(default_data[['Motor power']])

        # Make predictions using the model
        predictions = price_prediction_model.predict(default_data)

        # Inverse transform the normalized predictions
        original_scale_predictions = price_scaler.inverse_transform(predictions.reshape(-1, 1))

        # Log data before prediction
        app.logger.info(f"Predicting with data: {default_data}")

        # Return the unnormalized predictions
        return jsonify({'predicted_price': original_scale_predictions.tolist()})

    except Exception as e:
        app.logger.error(f"Error: {str(e)}")
        return jsonify({'error': f'Internal Server Error: {str(e)}'})

In [22]:
@app.route('/')
def home():
    return "hello"

In [23]:
if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Dec/2023 10:02:18] "POST /predict HTTP/1.1" 200 -
